Imports packages and connection to internal Actuary Team database

In [1]:
import pandas as pd
import pyodbc
connection_string = ("DRIVER={SQL Server};"
"SERVER=USDF11V0954;"
"DATABASE=Actuarial_AH;"
"Trust_Connection=yes;")

conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

Runs SQL query that creates a cohort of individuals that had a script for Wegovy, Saxenda, or Zepbound between service dates 2022 and 2025. Using that cohort, returns medical claims dataset that were paid between 2021 and 2025 and were actively enrolled since 2021.

In [70]:
med = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEDICAL_CLAIM_ID, 
       M.MEMBER_ID, 
       M.PAID_DATE, 
	   YEAR(M.PAID_DATE) AS PAID_YEAR,
       M.SERVICE_DATE,
YEAR(M.SERVICE_DATE) AS SERVICE_YEAR,
       M.MEDICAL_PAID_AMOUNT, 
       M.IS_TELEMEDICINE, 
       M.IS_ER_AVOIDABLE, 
M.DIAGNOSIS_DESC_ICD10_1 AS PRIMARY_DX,
       M.PROCEDURE_DESC, 
       M.ARTTOS_V2_L1, 
       M.ARTTOS_V2_L3, 
M.DX_IS_CHRONIC,
       M.ICD10_CHAPTER, 
       M.ICD10_CATEGORY, 
       M.ICD10_SECTION, 
       M.MEG_EPISODE_DESCRIPTION, 
       M.OP_SURG_INC,
       M.IS_PCP_VISIT, 
       M.INCLUDED_SPECIALIST, 
       M.ER_VISIT_FLAG, 
       M.IS_URGENT_CARE_VISIT, 
       M.IS_PREVENTIVE_VISIT, 
       M.IP_ADMIT_INC
FROM [Actuarial_AH].[dbo].[SN_Medical] M
JOIN 
Actuarial_AH.dbo.SN_Member MB ON M.MEMBER_ID = MB.MEMBER_ID
WHERE M.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND MB.MONTH_KEY = '2024-12-01' 
AND MB.MEDICAL_ENROLLMENT_STATUS = 3
AND MB.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND MB.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
"""
med = pd.read_sql(med, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_30428\2598388549.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  med = pd.read_sql(med, conn)


In [1]:
#create mock claims dataset to replace company data
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set seed for reproducibility
np.random.seed(42)

# Parameters
num_records = 100000
num_members = 10000

# Generate member IDs (weight loss cohort)
member_ids = np.random.choice(range(10000, 20000), size=num_members, replace=False)
selected_member_ids = np.random.choice(member_ids, size=num_records, replace=True)

# Generate random dates between 2021 and 2024
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)
paid_dates = [random_date(start_date, end_date) for _ in range(num_records)]
service_dates = [random_date(start_date, end_date) for _ in range(num_records)]

# Claim IDs
medical_claim_ids = np.arange(1, num_records + 1)

# Amounts and binary flags
medical_paid_amounts = np.round(np.random.exponential(scale=200, size=num_records), 2)
bool_choices = [0, 1]

# Text-based columns (mocked)
diagnosis_desc = np.random.choice(['Obesity', 'Type 2 Diabetes', 'Hypertension', 'Hyperlipidemia'], size=num_records)
procedure_desc = np.random.choice(['Office Visit', 'Lab Test', 'Imaging', 'Bariatric Surgery'], size=num_records)
arttos_l1 = np.random.choice(['Evaluation & Management', 'Surgery', 'Radiology'], size=num_records)
arttos_l3 = np.random.choice(['Office Visit', 'Major Surgery', 'Outpatient Imaging'], size=num_records)
icd10_chapter = np.random.choice(['Endocrine', 'Circulatory', 'Digestive'], size=num_records)
icd10_category = np.random.choice(['E11', 'I10', 'K21'], size=num_records)
icd10_section = np.random.choice(['E11.9', 'I10.0', 'K21.9'], size=num_records)
meg_description = np.random.choice(['Diabetes Management', 'Obesity Treatment', 'Routine Checkup'], size=num_records)

# Create DataFrame
med = pd.DataFrame({
    'MEDICAL_CLAIM_ID': medical_claim_ids,
    'MEMBER_ID': selected_member_ids,
    'PAID_DATE': paid_dates,
    'PAID_YEAR': [d.year for d in paid_dates],
    'SERVICE_DATE': service_dates,
    'SERVICE_YEAR': [d.year for d in service_dates],
    'MEDICAL_PAID_AMOUNT': medical_paid_amounts,
    'IS_TELEMEDICINE': np.random.choice(bool_choices, size=num_records),
    'IS_ER_AVOIDABLE': np.random.choice(bool_choices, size=num_records),
    'PRIMARY_DX': diagnosis_desc,
    'PROCEDURE_DESC': procedure_desc,
    'ARTTOS_V2_L1': arttos_l1,
    'ARTTOS_V2_L3': arttos_l3,
    'DX_IS_CHRONIC': np.random.choice(bool_choices, size=num_records),
    'ICD10_CHAPTER': icd10_chapter,
    'ICD10_CATEGORY': icd10_category,
    'ICD10_SECTION': icd10_section,
    'MEG_EPISODE_DESCRIPTION': meg_description,
    'OP_SURG_INC': np.random.choice(bool_choices, size=num_records),
    'IS_PCP_VISIT': np.random.choice(bool_choices, size=num_records),
    'INCLUDED_SPECIALIST': np.random.choice(bool_choices, size=num_records),
    'ER_VISIT_FLAG': np.random.choice(bool_choices, size=num_records),
    'IS_URGENT_CARE_VISIT': np.random.choice(bool_choices, size=num_records),
    'IS_PREVENTIVE_VISIT': np.random.choice(bool_choices, size=num_records),
    'IP_ADMIT_INC': np.random.choice(bool_choices, size=num_records)
})

med.head()


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1,18180,2024-03-25,2024,2022-11-17,2022,116.36,1,0,Hyperlipidemia,...,I10,E11.9,Routine Checkup,1,1,0,0,1,1,1
1,2,13277,2024-02-17,2024,2023-05-05,2023,96.93,0,1,Hyperlipidemia,...,K21,E11.9,Diabetes Management,1,1,0,0,0,1,1
2,3,12610,2021-11-05,2021,2021-09-15,2021,92.08,0,0,Hypertension,...,I10,E11.9,Obesity Treatment,0,1,0,0,1,0,1
3,4,18708,2022-03-16,2022,2023-12-11,2023,256.49,1,0,Hypertension,...,K21,I10.0,Diabetes Management,1,1,1,0,0,0,0
4,5,16682,2024-02-10,2024,2022-11-09,2022,141.06,1,0,Hyperlipidemia,...,I10,I10.0,Diabetes Management,0,0,0,0,1,0,0


Runs SQL query that creates a cohort of individuals that had a script for Wegovy, Saxenda, or Zepbound between service dates 2022 and 2025. Using that cohort, returns pharmacy claims dataset that were paid between 2021 and 2025 and were actively enrolled since 2021.

In [71]:
rx = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE BETWEEN '2022-01-01' AND '2024-12-31'
)
SELECT R.RX_CLAIM_ID, 
       R.MEMBER_ID, 
       R.PAID_DATE,
       R.DAYS,
       R.SERVICE_DATE, 
       R.DRUG_NAME_GENERIC,
	   R.DRUG_NAME_PREFERRED,
	   R.RX_PAID_AMOUNT,
	   R.RX_SCRIPT_COUNT,
	   R.ART_DRUGGRPS_L1,
	   R.ART_DRUGGRPS_L2,
	   YEAR(R.PAID_DATE) AS PAID_YEAR,
YEAR(R.SERVICE_DATE) AS SERVICE_YEAR
FROM [Actuarial_AH].[dbo].[SN_Rx] R
JOIN 
Actuarial_AH.dbo.SN_Member M ON R.MEMBER_ID = M.MEMBER_ID
WHERE R.PAID_DATE BETWEEN '2021-01-01' AND '2024-12-31'
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
AND M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
"""
rx = pd.read_sql(rx, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_30428\3517386714.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rx = pd.read_sql(rx, conn)


In [2]:
# Set seed for reproducibility
np.random.seed(42)

# Use the same MEMBER_IDs as the medical dataset (assumed already created)
# Simulate cohort of members
rx_member_ids = np.random.choice(range(10000, 20000), size=10000, replace=False)
num_rx_records = 100000
selected_rx_member_ids = np.random.choice(rx_member_ids, size=num_rx_records, replace=True)

# Date range
start_date = datetime(2021, 1, 1)
end_date = datetime(2024, 12, 31)

def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

rx_paid_dates = [random_date(start_date, end_date) for _ in range(num_rx_records)]
rx_service_dates = [random_date(datetime(2022, 1, 1), datetime(2024, 12, 31)) for _ in range(num_rx_records)]

# Random drug assignment (cohort drugs + others)
cohort_drugs = [
    ('semaglutide', 'Wegovy'),
    ('liraglutide', 'Saxenda'),
    ('tirzepatide', 'Zepbound')
]

other_drugs = [
    ('metformin', 'Glucophage'),
    ('atorvastatin', 'Lipitor'),
    ('lisinopril', 'Prinivil'),
    ('amlodipine', 'Norvasc'),
    ('hydrochlorothiazide', 'Microzide'),
    ('levothyroxine', 'Synthroid'),
    ('albuterol', 'Ventolin'),
]

# Mix 60% cohort drugs, 40% other
cohort_size = int(num_rx_records * 0.6)
other_size = num_rx_records - cohort_size
drug_mix = random.choices(cohort_drugs, k=cohort_size) + random.choices(other_drugs, k=other_size)
random.shuffle(drug_mix)  # Shuffle to mix cohort and non-cohort drugs

generic_names, preferred_names = zip(*drug_mix)

# Other fields
rx_claim_ids = np.arange(1, num_rx_records + 1)
rx_days = np.random.choice([30, 60, 90], size=num_rx_records)
rx_paid_amounts = np.round(np.random.exponential(scale=150, size=num_rx_records), 2)
rx_script_counts = np.random.choice([1, 2, 3], size=num_rx_records)
art_druggrps_l1 = np.random.choice(['Antidiabetics', 'Obesity Agents', 'Antihypertensives', 'Respiratory Agents'], size=num_rx_records)
art_druggrps_l2 = np.random.choice(['GLP-1 Agonists', 'ACE Inhibitors', 'Beta Blockers', 'Inhalers'], size=num_rx_records)

# Build DataFrame
rx = pd.DataFrame({
    'RX_CLAIM_ID': rx_claim_ids,
    'MEMBER_ID': selected_rx_member_ids,
    'PAID_DATE': rx_paid_dates,
    'DAYS': rx_days,
    'SERVICE_DATE': rx_service_dates,
    'DRUG_NAME_GENERIC': generic_names,
    'DRUG_NAME_PREFERRED': preferred_names,
    'RX_PAID_AMOUNT': rx_paid_amounts,
    'RX_SCRIPT_COUNT': rx_script_counts,
    'ART_DRUGGRPS_L1': art_druggrps_l1,
    'ART_DRUGGRPS_L2': art_druggrps_l2,
    'PAID_YEAR': [d.year for d in rx_paid_dates],
    'SERVICE_YEAR': [d.year for d in rx_service_dates]
})

# Preview
rx.head()


,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,1,18180,2022-11-27,60,2023-11-29,semaglutide,Wegovy,31.73,3,Antidiabetics,ACE Inhibitors,2022,2023
1,2,13277,2024-05-18,30,2022-05-02,tirzepatide,Zepbound,11.59,1,Obesity Agents,ACE Inhibitors,2024,2022
2,3,12610,2021-02-16,60,2023-10-08,liraglutide,Saxenda,46.64,2,Obesity Agents,ACE Inhibitors,2021,2023
3,4,18708,2024-04-17,60,2024-08-05,tirzepatide,Zepbound,43.10,3,Obesity Agents,Beta Blockers,2024,2024
4,5,16682,2021-12-14,30,2022-12-31,metformin,Glucophage,131.72,2,Antihypertensives,ACE Inhibitors,2021,2022


Runs SQL query that creates a cohort of individuals that had a script for Wegovy, Saxenda, or Zepbound between service dates 2022 and 2025. Using that cohort, returns unique members dataset that were actively enrolled since 2021.

In [72]:
member = """
WITH WEIGHTLOSSCOHORT1 AS (
    SELECT DISTINCT MEMBER_ID,
           YEAR(SERVICE_DATE) AS PRESCRIPTION_YEAR
    FROM Actuarial_AH.DBO.SN_Rx
    WHERE DRUG_NAME_PREFERRED IN ('Wegovy', 'Saxenda', 'Zepbound')
    AND SERVICE_DATE >= '2022-01-01'
    AND SERVICE_DATE < '2025-01-01'
)
SELECT M.MEMBER_ID,
       M.MEMBER_STATUS,
       M.HOME_STATE,
	   CASE 
    WHEN M.MEMBER_RELATIONSHIP = 0 THEN 'Subscriber'
    WHEN M.MEMBER_RELATIONSHIP = 1 THEN 'Spouse'
    WHEN M.MEMBER_RELATIONSHIP = 2 THEN 'Dependent'
    WHEN M.MEMBER_RELATIONSHIP = 3 THEN 'Domestic Partner'
    WHEN M.MEMBER_RELATIONSHIP = 4 THEN 'Unknown'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS MEMBER_RELATIONSHIP,
CASE 
        WHEN M.GENDER = 0 THEN 'Female'
        WHEN M.GENDER = 1 THEN 'Male'
        WHEN M.GENDER = 2 THEN 'Unknown'
        ELSE 'Unknown' 
    END AS GENDER,
    DATEDIFF(YEAR, CONVERT(DATE, M.BIRTH_YEAR_MO + '-01'), GETDATE()) - 
    CASE 
        WHEN MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > MONTH(GETDATE()) 
             OR (MONTH(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) = MONTH(GETDATE()) 
                 AND DAY(CONVERT(DATE, M.BIRTH_YEAR_MO + '-01')) > DAY(GETDATE())) 
        THEN 1 
        ELSE 0 
    END AS Age,
	CASE 
    WHEN M.BIRTH_YEAR >= 1946 AND M.BIRTH_YEAR < 1965 THEN 'Baby Boomers'
	WHEN M.BIRTH_YEAR >= 1965 AND M.BIRTH_YEAR < 1981 THEN 'Generation X'
	WHEN M.BIRTH_YEAR >= 1981 AND M.BIRTH_YEAR < 1997 THEN 'Millenials'
	WHEN M.BIRTH_YEAR >= 1997 AND M.BIRTH_YEAR < 2012 THEN 'Generation Z'
    ELSE 'Unknown' -- Optional: Handle unexpected values
END AS GENERATIONS
FROM [Actuarial_AH].[dbo].[SN_MEMBER] M
WHERE M.MEMBER_ID IN (
    SELECT MEMBER_ID
    FROM WEIGHTLOSSCOHORT1
    GROUP BY MEMBER_ID
	)
AND M.MONTH_KEY = '2024-12-01' 
AND M.MEDICAL_ENROLLMENT_STATUS = 3
AND M.MEDICAL_CONT_ENRLMNT_START <= '2021-01-01'
"""
member = pd.read_sql(member, conn)

C:\Users\U1229773\AppData\Local\Temp\ipykernel_30428\1131752232.py:55: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  member = pd.read_sql(member, conn)


In [3]:
# Set seed for reproducibility
np.random.seed(42)

# Assume these MEMBER_IDs come from your medical/pharmacy datasets
member_ids = np.random.choice(range(10000, 20000), size=10000, replace=False)
num_members = len(member_ids)

# Mock MEMBER_STATUS (1 = active, 2 = COBRA, 3 = retired)
member_status = np.random.choice([1, 2, 3], size=num_members)

# HOME_STATE (US states)
states = ['UT', 'CA', 'TX', 'NY', 'FL', 'WA', 'IL', 'CO', 'PA', 'NC']
home_states = np.random.choice(states, size=num_members)

# MEMBER_RELATIONSHIP codes
relationship_codes = np.random.choice([0, 1, 2, 3, 4], size=num_members)
relationship_map = {
    0: 'Subscriber',
    1: 'Spouse',
    2: 'Dependent',
    3: 'Domestic Partner',
    4: 'Unknown'
}
relationship_labels = [relationship_map[r] for r in relationship_codes]

# GENDER codes
genders = np.random.choice([0, 1, 2], size=num_members)
gender_map = {
    0: 'Female',
    1: 'Male',
    2: 'Unknown'
}
gender_labels = [gender_map[g] for g in genders]

# BIRTH_YEAR_MO generation
birth_years = np.random.choice(range(1946, 2012), size=num_members)
birth_months = np.random.choice(range(1, 13), size=num_members)
birth_year_mo = [f"{y}-{m:02d}" for y, m in zip(birth_years, birth_months)]

# Age calculation as of June 28, 2025
ref_date = datetime(2025, 6, 28)
ages = []
generations = []

for year, month in zip(birth_years, birth_months):
    dob = datetime(year, month, 1)
    age = ref_date.year - dob.year - ((ref_date.month, ref_date.day) < (dob.month, dob.day))
    ages.append(age)

    if 1946 <= year < 1965:
        generations.append('Baby Boomers')
    elif 1965 <= year < 1981:
        generations.append('Generation X')
    elif 1981 <= year < 1997:
        generations.append('Millenials')
    elif 1997 <= year < 2012:
        generations.append('Generation Z')
    else:
        generations.append('Unknown')

# Create DataFrame
member = pd.DataFrame({
    'MEMBER_ID': member_ids,
    'MEMBER_STATUS': member_status,
    'HOME_STATE': home_states,
    'MEMBER_RELATIONSHIP': relationship_labels,
    'GENDER': gender_labels,
    'Age': ages,
    'GENERATIONS': generations
})

# Preview
member.head()

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,16252,2,CA,Subscriber,Male,56,Generation X
1,14684,3,NC,Domestic Partner,Male,73,Baby Boomers
2,11731,2,WA,Spouse,Unknown,71,Baby Boomers
3,14742,3,FL,Subscriber,Female,55,Generation X
4,14521,1,FL,Subscriber,Unknown,39,Millenials


Reviewing medical, pharmacy and member tables

In [4]:
med

,MEDICAL_CLAIM_ID,MEMBER_ID,PAID_DATE,PAID_YEAR,SERVICE_DATE,SERVICE_YEAR,MEDICAL_PAID_AMOUNT,IS_TELEMEDICINE,IS_ER_AVOIDABLE,PRIMARY_DX,...,ICD10_CATEGORY,ICD10_SECTION,MEG_EPISODE_DESCRIPTION,OP_SURG_INC,IS_PCP_VISIT,INCLUDED_SPECIALIST,ER_VISIT_FLAG,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,1,18180,2024-03-25,2024,2022-11-17,2022,116.36,1,0,Hyperlipidemia,...,I10,E11.9,Routine Checkup,1,1,0,0,1,1,1
1,2,13277,2024-02-17,2024,2023-05-05,2023,96.93,0,1,Hyperlipidemia,...,K21,E11.9,Diabetes Management,1,1,0,0,0,1,1
2,3,12610,2021-11-05,2021,2021-09-15,2021,92.08,0,0,Hypertension,...,I10,E11.9,Obesity Treatment,0,1,0,0,1,0,1
3,4,18708,2022-03-16,2022,2023-12-11,2023,256.49,1,0,Hypertension,...,K21,I10.0,Diabetes Management,1,1,1,0,0,0,0
4,5,16682,2024-02-10,2024,2022-11-09,2022,141.06,1,0,Hyperlipidemia,...,I10,I10.0,Diabetes Management,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,12607,2021-05-28,2021,2021-08-14,2021,44.26,1,1,Hypertension,...,K21,I10.0,Obesity Treatment,0,1,1,1,1,0,1
99996,99997,14528,2022-01-05,2022,2024-08-01,2024,69.52,1,1,Type 2 Diabetes,...,E11,E11.9,Obesity Treatment,0,0,1,0,1,0,0
99997,99998,14138,2023-05-01,2023,2024-02-01,2024,426.73,1,1,Type 2 Diabetes,...,I10,K21.9,Obesity Treatment,0,0,1,1,1,1,1
99998,99999,16528,2022-09-25,2022,2022-10-08,2022,83.67,0,1,Obesity,...,I10,E11.9,Routine Checkup,1,1,1,0,0,0,0


In [5]:
rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR
0,1,18180,2022-11-27,60,2023-11-29,semaglutide,Wegovy,31.73,3,Antidiabetics,ACE Inhibitors,2022,2023
1,2,13277,2024-05-18,30,2022-05-02,tirzepatide,Zepbound,11.59,1,Obesity Agents,ACE Inhibitors,2024,2022
2,3,12610,2021-02-16,60,2023-10-08,liraglutide,Saxenda,46.64,2,Obesity Agents,ACE Inhibitors,2021,2023
3,4,18708,2024-04-17,60,2024-08-05,tirzepatide,Zepbound,43.10,3,Obesity Agents,Beta Blockers,2024,2024
4,5,16682,2021-12-14,30,2022-12-31,metformin,Glucophage,131.72,2,Antihypertensives,ACE Inhibitors,2021,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,12607,2024-01-09,60,2024-02-24,tirzepatide,Zepbound,456.91,2,Antidiabetics,Beta Blockers,2024,2024
99996,99997,14528,2023-11-05,30,2023-05-03,tirzepatide,Zepbound,410.99,1,Antidiabetics,GLP-1 Agonists,2023,2023
99997,99998,14138,2022-06-01,30,2023-10-31,semaglutide,Wegovy,236.02,2,Obesity Agents,Beta Blockers,2022,2023
99998,99999,16528,2024-11-09,90,2022-07-09,liraglutide,Saxenda,82.51,3,Respiratory Agents,Inhalers,2024,2022


In [6]:
member

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS
0,16252,2,CA,Subscriber,Male,56,Generation X
1,14684,3,NC,Domestic Partner,Male,73,Baby Boomers
2,11731,2,WA,Spouse,Unknown,71,Baby Boomers
3,14742,3,FL,Subscriber,Female,55,Generation X
4,14521,1,FL,Subscriber,Unknown,39,Millenials
...,...,...,...,...,...,...,...
9995,15734,2,WA,Subscriber,Unknown,46,Generation X
9996,15191,3,CA,Dependent,Male,15,Generation Z
9997,15390,2,NY,Subscriber,Unknown,43,Millenials
9998,10860,1,UT,Dependent,Unknown,42,Millenials


Find members that had taken a weight-loss GLP-1 in 2020 and 2021 and remove from the tables. This study looks at individuals who initiated in 2022.

In [7]:
# Define the drugs to filter out
drugs_to_remove = ['Wegovy', 'Saxenda', 'Zepbound']

# Filter to find members with the specified drugs in 2021
members_to_remove = rx[((rx['SERVICE_YEAR'] == 2021) | (rx['SERVICE_YEAR'] == 2020)) & (rx['DRUG_NAME_PREFERRED'].isin(drugs_to_remove))]['MEMBER_ID'].unique()

# Remove those members from the original DataFrame
rx_members_glp_22_24 = rx[~rx['MEMBER_ID'].isin(members_to_remove)]
med_members_glp_22_24 = med[~med['MEMBER_ID'].isin(members_to_remove)]
members_glp_22_24 = member[~member['MEMBER_ID'].isin(members_to_remove)]

Create a view that will set up the Percentage of Days Covered (PDC) metric. Locate GLP-1s in the rx dataset and determine days filled, start and end days per script.

In [8]:
# Define NDCs or names of GLP-1 drugs
glp1_drugs = ['Wegovy', 'Saxenda', 'Zepbound'] 

# Step 1: Filter for GLP-1 claims only
glp1_drugs_rx = rx_members_glp_22_24[rx_members_glp_22_24['DRUG_NAME_PREFERRED'].isin(glp1_drugs)].copy()

# Step 2: Convert fill_date to datetime
glp1_drugs_rx['SERVICE_DATE'] = pd.to_datetime(glp1_drugs_rx['SERVICE_DATE'])

# Step 3: Create a column for days covered per fill
glp1_drugs_rx['END_DATE'] = glp1_drugs_rx['SERVICE_DATE'] + pd.to_timedelta(glp1_drugs_rx['DAYS'] - 1, unit='D')
glp1_drugs_rx

,RX_CLAIM_ID,MEMBER_ID,PAID_DATE,DAYS,SERVICE_DATE,DRUG_NAME_GENERIC,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT,RX_SCRIPT_COUNT,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,PAID_YEAR,SERVICE_YEAR,END_DATE
0,1,18180,2022-11-27,60,2023-11-29,semaglutide,Wegovy,31.73,3,Antidiabetics,ACE Inhibitors,2022,2023,2024-01-27
1,2,13277,2024-05-18,30,2022-05-02,tirzepatide,Zepbound,11.59,1,Obesity Agents,ACE Inhibitors,2024,2022,2022-05-31
2,3,12610,2021-02-16,60,2023-10-08,liraglutide,Saxenda,46.64,2,Obesity Agents,ACE Inhibitors,2021,2023,2023-12-06
3,4,18708,2024-04-17,60,2024-08-05,tirzepatide,Zepbound,43.10,3,Obesity Agents,Beta Blockers,2024,2024,2024-10-03
8,9,19325,2021-02-13,30,2023-02-01,liraglutide,Saxenda,7.83,3,Antihypertensives,Beta Blockers,2021,2023,2023-03-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99996,12607,2024-01-09,60,2024-02-24,tirzepatide,Zepbound,456.91,2,Antidiabetics,Beta Blockers,2024,2024,2024-04-23
99996,99997,14528,2023-11-05,30,2023-05-03,tirzepatide,Zepbound,410.99,1,Antidiabetics,GLP-1 Agonists,2023,2023,2023-06-01
99997,99998,14138,2022-06-01,30,2023-10-31,semaglutide,Wegovy,236.02,2,Obesity Agents,Beta Blockers,2022,2023,2023-11-29
99998,99999,16528,2024-11-09,90,2022-07-09,liraglutide,Saxenda,82.51,3,Respiratory Agents,Inhalers,2024,2022,2022-10-06


Aggregate sum of days filled by member ID and earliest and latest service/fill dates. Determine 3 respective years from earliest service date to create a 365-day timelines to calculate PDC.

In [9]:
member_pdc = glp1_drugs_rx.groupby('MEMBER_ID').agg(
    min_service_date=('SERVICE_DATE', 'min'),
    max_end_date=('END_DATE', 'max'),
    covered_days = ('DAYS', 'sum')
).reset_index()

# Add 12 months for each year
for year in range(1, 4):
    member_pdc[f'min_service_date_year_{year}'] = member_pdc['min_service_date'] + pd.DateOffset(months=12 * year)

member_pdc

,MEMBER_ID,min_service_date,max_end_date,covered_days,min_service_date_year_1,min_service_date_year_2,min_service_date_year_3
0,10000,2022-05-05,2024-10-06,240,2023-05-05,2024-05-05,2025-05-05
1,10001,2022-10-26,2024-12-27,480,2023-10-26,2024-10-26,2025-10-26
2,10002,2022-04-23,2024-09-16,330,2023-04-23,2024-04-23,2025-04-23
3,10003,2022-01-26,2024-06-24,450,2023-01-26,2024-01-26,2025-01-26
4,10004,2022-02-09,2025-03-07,690,2023-02-09,2024-02-09,2025-02-09
...,...,...,...,...,...,...,...
9972,19995,2022-10-30,2024-10-16,150,2023-10-30,2024-10-30,2025-10-30
9973,19996,2022-04-10,2025-01-30,240,2023-04-10,2024-04-10,2025-04-10
9974,19997,2022-05-02,2025-01-21,720,2023-05-02,2024-05-02,2025-05-02
9975,19998,2022-01-04,2024-09-17,360,2023-01-04,2024-01-04,2025-01-04


Only keep members that had earliest script dates in 2022.

In [10]:
members_2022 = member_pdc[member_pdc['min_service_date'] < '2023']
members_2022

,MEMBER_ID,min_service_date,max_end_date,covered_days,min_service_date_year_1,min_service_date_year_2,min_service_date_year_3
0,10000,2022-05-05,2024-10-06,240,2023-05-05,2024-05-05,2025-05-05
1,10001,2022-10-26,2024-12-27,480,2023-10-26,2024-10-26,2025-10-26
2,10002,2022-04-23,2024-09-16,330,2023-04-23,2024-04-23,2025-04-23
3,10003,2022-01-26,2024-06-24,450,2023-01-26,2024-01-26,2025-01-26
4,10004,2022-02-09,2025-03-07,690,2023-02-09,2024-02-09,2025-02-09
...,...,...,...,...,...,...,...
9972,19995,2022-10-30,2024-10-16,150,2023-10-30,2024-10-30,2025-10-30
9973,19996,2022-04-10,2025-01-30,240,2023-04-10,2024-04-10,2025-04-10
9974,19997,2022-05-02,2025-01-21,720,2023-05-02,2024-05-02,2025-05-02
9975,19998,2022-01-04,2024-09-17,360,2023-01-04,2024-01-04,2025-01-04


In [11]:
# Initialize a dictionary to store results
results = []

# Loop through each member and calculate covered days
for index, member in members_2022.iterrows():
    member_id = member['MEMBER_ID']
    
    # Filter rx_df for claims before year 1 and year 2
    covered_days_year_1 = glp1_drugs_rx[(glp1_drugs_rx['MEMBER_ID'] == member_id) & 
                                 (glp1_drugs_rx['SERVICE_DATE'] < member['min_service_date_year_1'])]['DAYS'].sum()
    
    covered_days_year_2 = glp1_drugs_rx[(glp1_drugs_rx['MEMBER_ID'] == member_id) & 
                                 ((glp1_drugs_rx['SERVICE_DATE'] > member['min_service_date_year_1']) & (glp1_drugs_rx['SERVICE_DATE'] < member['min_service_date_year_2']))]['DAYS'].sum()
    
    # Append results
    results.append({
        'MEMBER_ID': member_id,
        'covered_days_year_1': covered_days_year_1,
        'covered_days_year_2': covered_days_year_2
    })

In [12]:
results_df = pd.DataFrame(results)
results_df

,MEMBER_ID,covered_days_year_1,covered_days_year_2
0,10000,60,90
1,10001,330,120
2,10002,90,150
3,10003,180,180
4,10004,210,210
...,...,...,...
8613,19995,30,120
8614,19996,90,0
8615,19997,270,300
8616,19998,210,30


In [13]:
members_2022_cvd = members_2022.merge(results_df, on='MEMBER_ID')
members_2022_cvd.drop(columns=['covered_days', 'min_service_date_year_3'], inplace=True)
members_2022_cvd['PDC1'] = members_2022_cvd['covered_days_year_1']/365
members_2022_cvd['PDC2'] = members_2022_cvd['covered_days_year_2']/365
members_2022_cvd

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
0,10000,2022-05-05,2024-10-06,2023-05-05,2024-05-05,60,90,0.164384,0.246575
1,10001,2022-10-26,2024-12-27,2023-10-26,2024-10-26,330,120,0.904110,0.328767
2,10002,2022-04-23,2024-09-16,2023-04-23,2024-04-23,90,150,0.246575,0.410959
3,10003,2022-01-26,2024-06-24,2023-01-26,2024-01-26,180,180,0.493151,0.493151
4,10004,2022-02-09,2025-03-07,2023-02-09,2024-02-09,210,210,0.575342,0.575342
...,...,...,...,...,...,...,...,...,...
8613,19995,2022-10-30,2024-10-16,2023-10-30,2024-10-30,30,120,0.082192,0.328767
8614,19996,2022-04-10,2025-01-30,2023-04-10,2024-04-10,90,0,0.246575,0.000000
8615,19997,2022-05-02,2025-01-21,2023-05-02,2024-05-02,270,300,0.739726,0.821918
8616,19998,2022-01-04,2024-09-17,2023-01-04,2024-01-04,210,30,0.575342,0.082192


In [14]:
members_2022_cvd.describe()

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
count,8618.000000,8618,8618,8618,8618,8618.000000,8618.000000,8618.000000,8618.000000
mean,15000.088071,2022-05-04 16:44:53.525179904,2024-09-02 19:05:45.045254144,2023-05-04 16:44:53.525179904,2024-05-04 09:08:03.731724288,179.213275,119.690183,0.490995,0.327918
min,10000.000000,2022-01-01 00:00:00,2022-03-03 00:00:00,2023-01-01 00:00:00,2024-01-01 00:00:00,30.000000,0.000000,0.082192,0.000000
25%,12508.250000,2022-02-14 00:00:00,2024-06-22 00:00:00,2023-02-14 00:00:00,2024-02-14 00:00:00,120.000000,60.000000,0.328767,0.164384
50%,14994.000000,2022-04-14 00:00:00,2024-10-26 00:00:00,2023-04-14 00:00:00,2024-04-14 00:00:00,180.000000,90.000000,0.493151,0.246575
75%,17500.750000,2022-07-09 00:00:00,2025-01-06 00:00:00,2023-07-09 00:00:00,2024-07-09 00:00:00,240.000000,180.000000,0.657534,0.493151
max,19999.000000,2022-12-31 00:00:00,2025-03-30 00:00:00,2023-12-31 00:00:00,2024-12-31 00:00:00,660.000000,630.000000,1.808219,1.726027
std,2892.290489,NaN,NaN,NaN,NaN,94.010156,90.989731,0.257562,0.249287


In [43]:
members_20 = members_2022_cvd[(members_2022_cvd['PDC1'] < 0.2) & (members_2022_cvd['PDC2'] < 0.2)]
members_20

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
46,10053,2022-01-13,2024-06-12,2023-01-13,2024-01-13,60,30,0.16,0.08
56,10065,2022-01-01,2024-04-17,2023-01-01,2024-01-01,30,0,0.08,0.00
90,10102,2022-01-20,2024-08-02,2023-01-20,2024-01-20,60,60,0.16,0.16
197,10225,2022-05-09,2024-09-03,2023-05-09,2024-05-09,60,0,0.16,0.00
232,10263,2022-02-28,2025-01-14,2023-02-28,2024-02-28,30,60,0.08,0.16
...,...,...,...,...,...,...,...,...,...
8497,19866,2022-03-17,2024-12-18,2023-03-17,2024-03-17,60,60,0.16,0.16
8515,19887,2022-04-28,2025-01-22,2023-04-28,2024-04-28,60,30,0.16,0.08
8549,19923,2022-04-09,2024-10-09,2023-04-09,2024-04-09,60,30,0.16,0.08
8550,19924,2022-02-08,2025-03-02,2023-02-08,2024-02-08,60,0,0.16,0.00


In [44]:
members_20.describe()

,MEMBER_ID,min_service_date,max_end_date,min_service_date_year_1,min_service_date_year_2,covered_days_year_1,covered_days_year_2,PDC1,PDC2
count,344.00,344,344,344,344,344.00,344.00,344.00,344.00
mean,14947.43,2022-05-03 10:27:54.418604544,2024-07-12 06:25:06.976744192,2023-05-03 10:27:54.418604544,2024-05-03 02:39:04.186046464,48.58,29.30,0.13,0.08
min,10053.00,2022-01-01 00:00:00,2022-03-03 00:00:00,2023-01-01 00:00:00,2024-01-01 00:00:00,30.00,0.00,0.08,0.00
25%,12631.25,2022-02-14 18:00:00,2024-05-05 00:00:00,2023-02-14 18:00:00,2024-02-14 18:00:00,30.00,0.00,0.08,0.00
50%,14897.00,2022-04-08 12:00:00,2024-10-27 00:00:00,2023-04-08 12:00:00,2024-04-08 12:00:00,60.00,30.00,0.16,0.08
75%,17233.00,2022-07-08 00:00:00,2025-01-09 12:00:00,2023-07-08 00:00:00,2024-07-08 00:00:00,60.00,60.00,0.16,0.16
max,19988.00,2022-12-27 00:00:00,2025-03-29 00:00:00,2023-12-27 00:00:00,2024-12-27 00:00:00,60.00,60.00,0.16,0.16
std,2774.14,NaN,NaN,NaN,NaN,14.59,26.41,0.04,0.07


In [17]:
med_agg = med_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='MEDICAL_PAID_AMOUNT', aggfunc='sum')
med_agg.reset_index(inplace=True)
med_agg.columns.name = None
med_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in med_agg.columns]
med_agg.rename(columns={'2021': 'Med_2021', '2022': 'Med_2022', '2023': 'Med_2023', '2024': 'Med_2024'}, inplace=True)
med_agg1 = med_agg
med_agg1

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,10000,504.49,361.46,110.85,291.18
1,10001,1387.75,99.18,540.72,1023.00
2,10002,322.38,297.95,417.92,335.61
3,10003,13.02,69.86,51.31,508.82
4,10004,112.38,1779.93,1332.14,689.23
...,...,...,...,...,...
9995,19995,341.72,141.79,186.30,19.10
9996,19996,NaN,217.26,58.46,328.93
9997,19997,1763.64,423.68,606.43,244.77
9998,19998,833.06,294.50,NaN,686.33


In [18]:
# Define inflation rates (as factors)
inflation_factors = {
    2021: 1.175, 
    2022: 1.117,   
    2023: 1.069, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in inflation_factors.items():
    column_name = f'Med_{year}'
    med_agg[column_name] = med_agg[column_name] * factor
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024
0,10000,592.77575,403.75082,118.49865,291.18
1,10001,1630.60625,110.78406,578.02968,1023.00
2,10002,378.79650,332.81015,446.75648,335.61
3,10003,15.29850,78.03362,54.85039,508.82
4,10004,132.04650,1988.18181,1424.05766,689.23
...,...,...,...,...,...
9995,19995,401.52100,158.37943,199.15470,19.10
9996,19996,NaN,242.67942,62.49374,328.93
9997,19997,2072.27700,473.25056,648.27367,244.77
9998,19998,978.84550,328.95650,NaN,686.33


In [19]:
med_agg['Med_Delta'] = med_agg['Med_2024'] - med_agg['Med_2021']
med_agg['Med_%_Change'] =(( med_agg['Med_2024'] - med_agg['Med_2021'])/ med_agg['Med_2021']) * 100
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change
0,10000,592.77575,403.75082,118.49865,291.18,-301.59575,-50.878557
1,10001,1630.60625,110.78406,578.02968,1023.00,-607.60625,-37.262598
2,10002,378.79650,332.81015,446.75648,335.61,-43.18650,-11.400977
3,10003,15.29850,78.03362,54.85039,508.82,493.52150,3225.946988
4,10004,132.04650,1988.18181,1424.05766,689.23,557.18350,421.960067
...,...,...,...,...,...,...,...
9995,19995,401.52100,158.37943,199.15470,19.10,-382.42100,-95.243088
9996,19996,NaN,242.67942,62.49374,328.93,NaN,NaN
9997,19997,2072.27700,473.25056,648.27367,244.77,-1827.50700,-88.188355
9998,19998,978.84550,328.95650,NaN,686.33,-292.51550,-29.883725


In [20]:
med_agg.fillna(0, inplace=True)

In [21]:
med_agg.isnull().sum()

MEMBER_ID       0
Med_2021        0
Med_2022        0
Med_2023        0
Med_2024        0
Med_Delta       0
Med_%_Change    0
dtype: int64

In [22]:
import numpy as np

# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in med_agg.iterrows():
    costs = np.array([row['Med_2021'], row['Med_2022'], row['Med_2023'], row['Med_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
med_agg['Med_Slope'] = slope_list
med_agg

,MEMBER_ID,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope
0,10000,592.77575,403.75082,118.49865,291.18,-301.59575,-50.878557,-119.003942
1,10001,1630.60625,110.78406,578.02968,1023.00,-607.60625,-37.262598,-135.557313
2,10002,378.79650,332.81015,446.75648,335.61,-43.18650,-11.400977,-1.561317
3,10003,15.29850,78.03362,54.85039,508.82,493.52150,3225.946988,145.738127
4,10004,132.04650,1988.18181,1424.05766,689.23,557.18350,421.960067,110.742635
...,...,...,...,...,...,...,...,...
9995,19995,401.52100,158.37943,199.15470,19.10,-382.42100,-95.243088,-110.648773
9996,19996,0.00000,242.67942,62.49374,328.93,0.00000,0.000000,80.660432
9997,19997,2072.27700,473.25056,648.27367,244.77,-1827.50700,-88.188355,-530.749789
9998,19998,978.84550,328.95650,0.00000,686.33,-292.51550,-29.883725,-120.650300


In [23]:
rx_agg = rx_members_glp_22_24.pivot_table(index='MEMBER_ID', columns='PAID_YEAR', values='RX_PAID_AMOUNT', aggfunc='sum')
rx_agg.reset_index(inplace=True)
rx_agg.columns.name = None
rx_agg.columns = [f"{col}" if col != 'PAID_YEAR' else 'PAID_YEAR' for col in rx_agg.columns]
rx_agg.rename(columns={'2021': 'Rx_2021', '2022': 'Rx_2022', '2023': 'Rx_2023', '2024': 'Rx_2024'}, inplace=True)
rx_agg1 = rx_agg
rx_agg1

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,10000,382.87,48.58,295.20,701.02
1,10001,398.02,189.47,407.75,142.92
2,10002,447.75,390.61,443.91,219.49
3,10003,129.24,314.75,228.73,782.66
4,10004,406.19,623.96,324.94,800.38
...,...,...,...,...,...
9995,19995,231.23,226.66,189.97,93.25
9996,19996,660.48,NaN,NaN,149.28
9997,19997,362.13,1241.17,69.37,469.68
9998,19998,142.15,244.64,494.70,574.96


In [24]:
# Define inflation rates (as factors)
rx_inflation_factors = {
    2021: 1.31, 
    2022: 1.20,   
    2023: 1.103, 
    2024: 1
}

# Normalize employer paid amounts for inflation
for year, factor in rx_inflation_factors.items():
    column_name = f'Rx_{year}'
    rx_agg[column_name] = rx_agg[column_name] * factor
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024
0,10000,501.5597,58.296,325.60560,701.02
1,10001,521.4062,227.364,449.74825,142.92
2,10002,586.5525,468.732,489.63273,219.49
3,10003,169.3044,377.700,252.28919,782.66
4,10004,532.1089,748.752,358.40882,800.38
...,...,...,...,...,...
9995,19995,302.9113,271.992,209.53691,93.25
9996,19996,865.2288,NaN,NaN,149.28
9997,19997,474.3903,1489.404,76.51511,469.68
9998,19998,186.2165,293.568,545.65410,574.96


In [25]:
rx_agg.fillna(0, inplace=True)
rx_agg.isnull().sum()

MEMBER_ID    0
Rx_2021      0
Rx_2022      0
Rx_2023      0
Rx_2024      0
dtype: int64

In [26]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [27]:
rx_agg['Rx_Delta'] = rx_agg['Rx_2024'] - rx_agg['Rx_2021']
rx_agg['Rx_%_Change'] =((rx_agg['Rx_2024'] - rx_agg['Rx_2021'])/ rx_agg['Rx_2021']) * 100
# Prepare the years and medical costs
years = np.array([2021, 2022, 2023, 2024])
slope_list = []

# Calculate the slope for each member
for index, row in rx_agg.iterrows():
    costs = np.array([row['Rx_2021'], row['Rx_2022'], row['Rx_2023'], row['Rx_2024']])
    # Perform linear regression to get the slope
    slope, intercept = np.polyfit(years, costs, 1)  # 1 indicates linear fit
    slope_list.append(slope)

# Add the slope to the DataFrame
rx_agg['Rx_Slope'] = slope_list
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,10000,501.56,58.30,325.61,701.02,199.46,39.77,86.57
1,10001,521.41,227.36,449.75,142.92,-378.49,-72.59,-91.31
2,10002,586.55,468.73,489.63,219.49,-367.06,-62.58,-108.03
3,10003,169.30,377.70,252.29,782.66,613.36,362.28,171.47
4,10004,532.11,748.75,358.41,800.38,268.27,50.42,41.45
...,...,...,...,...,...,...,...,...
9995,19995,302.91,271.99,209.54,93.25,-209.66,-69.22,-69.14
9996,19996,865.23,0.00,0.00,149.28,-715.95,-82.75,-214.78
9997,19997,474.39,1489.40,76.52,469.68,-4.71,-0.99,-142.70
9998,19998,186.22,293.57,545.65,574.96,388.74,208.76,141.83


In [28]:
rx_agg.replace(np.inf, 0, inplace=True)
rx_agg

,MEMBER_ID,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,10000,501.56,58.30,325.61,701.02,199.46,39.77,86.57
1,10001,521.41,227.36,449.75,142.92,-378.49,-72.59,-91.31
2,10002,586.55,468.73,489.63,219.49,-367.06,-62.58,-108.03
3,10003,169.30,377.70,252.29,782.66,613.36,362.28,171.47
4,10004,532.11,748.75,358.41,800.38,268.27,50.42,41.45
...,...,...,...,...,...,...,...,...
9995,19995,302.91,271.99,209.54,93.25,-209.66,-69.22,-69.14
9996,19996,865.23,0.00,0.00,149.28,-715.95,-82.75,-214.78
9997,19997,474.39,1489.40,76.52,469.68,-4.71,-0.99,-142.70
9998,19998,186.22,293.57,545.65,574.96,388.74,208.76,141.83


In [29]:
member_mrg = members_glp_22_24.merge(med_agg, on='MEMBER_ID')
member_mrg = member_mrg.merge(rx_agg, on='MEMBER_ID')                                     
member_mrg                                    

,MEMBER_ID,MEMBER_STATUS,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,...,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,16252,2,CA,Subscriber,Male,56,Generation X,1194.55,309.93,913.87,...,-816.66,-68.37,-184.61,1409.72,26.82,857.60,538.23,-871.49,-61.82,-178.37
1,14684,3,NC,Domestic Partner,Male,73,Baby Boomers,665.93,803.29,1258.23,...,236.26,35.48,116.37,407.65,184.13,852.82,840.13,432.48,106.09,196.61
2,11731,2,WA,Spouse,Unknown,71,Baby Boomers,0.00,229.47,184.58,...,0.00,0.00,237.97,197.14,65.56,287.83,532.90,335.76,170.31,122.95
3,14742,3,FL,Subscriber,Female,55,Generation X,786.49,1075.99,442.21,...,156.56,19.91,-16.41,438.93,322.56,794.38,315.10,-123.83,-28.21,10.03
4,14521,1,FL,Subscriber,Unknown,39,Millenials,321.49,92.41,461.79,...,-200.45,-62.35,-23.20,498.27,661.74,523.32,410.82,-87.45,-17.55,-40.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,2,WA,Subscriber,Unknown,46,Generation X,719.24,117.49,14.26,...,-379.90,-52.82,-124.29,461.76,809.26,425.11,245.43,-216.33,-46.85,-103.31
9996,15191,3,CA,Dependent,Male,15,Generation Z,211.22,557.73,505.10,...,0.00,0.00,-68.63,854.34,105.83,490.50,804.78,-49.56,-5.80,23.60
9997,15390,2,NY,Subscriber,Unknown,43,Millenials,587.74,109.96,358.81,...,-315.10,-53.61,-69.64,440.61,32.77,167.16,239.64,-200.97,-45.61,-46.85
9998,10860,1,UT,Dependent,Unknown,42,Millenials,27.67,538.65,176.45,...,221.87,801.80,30.34,91.20,421.44,155.13,257.72,166.52,182.58,23.32


In [30]:
member1 = member_mrg.drop(columns=['MEMBER_STATUS'])
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,16252,CA,Subscriber,Male,56,Generation X,1194.55,309.93,913.87,377.89,-816.66,-68.37,-184.61,1409.72,26.82,857.60,538.23,-871.49,-61.82,-178.37
1,14684,NC,Domestic Partner,Male,73,Baby Boomers,665.93,803.29,1258.23,902.19,236.26,35.48,116.37,407.65,184.13,852.82,840.13,432.48,106.09,196.61
2,11731,WA,Spouse,Unknown,71,Baby Boomers,0.00,229.47,184.58,808.18,0.00,0.00,237.97,197.14,65.56,287.83,532.90,335.76,170.31,122.95
3,14742,FL,Subscriber,Female,55,Generation X,786.49,1075.99,442.21,943.05,156.56,19.91,-16.41,438.93,322.56,794.38,315.10,-123.83,-28.21,10.03
4,14521,FL,Subscriber,Unknown,39,Millenials,321.49,92.41,461.79,121.04,-200.45,-62.35,-23.20,498.27,661.74,523.32,410.82,-87.45,-17.55,-40.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,WA,Subscriber,Unknown,46,Generation X,719.24,117.49,14.26,339.34,-379.90,-52.82,-124.29,461.76,809.26,425.11,245.43,-216.33,-46.85,-103.31
9996,15191,CA,Dependent,Male,15,Generation Z,211.22,557.73,505.10,0.00,0.00,0.00,-68.63,854.34,105.83,490.50,804.78,-49.56,-5.80,23.60
9997,15390,NY,Subscriber,Unknown,43,Millenials,587.74,109.96,358.81,272.64,-315.10,-53.61,-69.64,440.61,32.77,167.16,239.64,-200.97,-45.61,-46.85
9998,10860,UT,Dependent,Unknown,42,Millenials,27.67,538.65,176.45,249.54,221.87,801.80,30.34,91.20,421.44,155.13,257.72,166.52,182.58,23.32


In [31]:
member1.HOME_STATE.value_counts()

HOME_STATE
NY    1051
IL    1030
CA    1005
FL     998
WA     997
UT     995
PA     993
TX     988
CO     985
NC     958
Name: count, dtype: int64

In [32]:
# Define the states to keep
states_to_keep = ['NC', 'PA', 'TX', 'NJ', 'MS', 'OH', 'MI']
member1['HOME_STATE'] = member1['HOME_STATE'].apply(lambda x: x if x in states_to_keep else 'OTHER')
member1

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
0,16252,OTHER,Subscriber,Male,56,Generation X,1194.55,309.93,913.87,377.89,-816.66,-68.37,-184.61,1409.72,26.82,857.60,538.23,-871.49,-61.82,-178.37
1,14684,NC,Domestic Partner,Male,73,Baby Boomers,665.93,803.29,1258.23,902.19,236.26,35.48,116.37,407.65,184.13,852.82,840.13,432.48,106.09,196.61
2,11731,OTHER,Spouse,Unknown,71,Baby Boomers,0.00,229.47,184.58,808.18,0.00,0.00,237.97,197.14,65.56,287.83,532.90,335.76,170.31,122.95
3,14742,OTHER,Subscriber,Female,55,Generation X,786.49,1075.99,442.21,943.05,156.56,19.91,-16.41,438.93,322.56,794.38,315.10,-123.83,-28.21,10.03
4,14521,OTHER,Subscriber,Unknown,39,Millenials,321.49,92.41,461.79,121.04,-200.45,-62.35,-23.20,498.27,661.74,523.32,410.82,-87.45,-17.55,-40.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,OTHER,Subscriber,Unknown,46,Generation X,719.24,117.49,14.26,339.34,-379.90,-52.82,-124.29,461.76,809.26,425.11,245.43,-216.33,-46.85,-103.31
9996,15191,OTHER,Dependent,Male,15,Generation Z,211.22,557.73,505.10,0.00,0.00,0.00,-68.63,854.34,105.83,490.50,804.78,-49.56,-5.80,23.60
9997,15390,OTHER,Subscriber,Unknown,43,Millenials,587.74,109.96,358.81,272.64,-315.10,-53.61,-69.64,440.61,32.77,167.16,239.64,-200.97,-45.61,-46.85
9998,10860,OTHER,Dependent,Unknown,42,Millenials,27.67,538.65,176.45,249.54,221.87,801.80,30.34,91.20,421.44,155.13,257.72,166.52,182.58,23.32


In [33]:
med.columns

Index(['MEDICAL_CLAIM_ID', 'MEMBER_ID', 'PAID_DATE', 'PAID_YEAR',
       'SERVICE_DATE', 'SERVICE_YEAR', 'MEDICAL_PAID_AMOUNT',
       'IS_TELEMEDICINE', 'IS_ER_AVOIDABLE', 'PRIMARY_DX', 'PROCEDURE_DESC',
       'ARTTOS_V2_L1', 'ARTTOS_V2_L3', 'DX_IS_CHRONIC', 'ICD10_CHAPTER',
       'ICD10_CATEGORY', 'ICD10_SECTION', 'MEG_EPISODE_DESCRIPTION',
       'OP_SURG_INC', 'IS_PCP_VISIT', 'INCLUDED_SPECIALIST', 'ER_VISIT_FLAG',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC'],
      dtype='object')

In [34]:
med_util = med.groupby('MEMBER_ID').agg({
    'DX_IS_CHRONIC': 'max',
    'ER_VISIT_FLAG': 'sum',
    'IS_PCP_VISIT': 'sum',
    'IS_URGENT_CARE_VISIT': 'sum',
    'IS_PREVENTIVE_VISIT': 'sum',
    'IP_ADMIT_INC': 'sum'
}).reset_index()
med_util

,MEMBER_ID,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,10000,1,5,5,5,5,4
1,10001,1,7,7,10,5,6
2,10002,1,6,5,7,6,6
3,10003,1,3,5,5,5,3
4,10004,1,8,4,11,8,8
...,...,...,...,...,...,...,...
9995,19995,1,4,5,6,4,5
9996,19996,1,4,5,4,2,3
9997,19997,1,9,5,11,8,4
9998,19998,1,5,4,7,3,5


In [35]:
episodes = med.ICD10_CHAPTER.value_counts()
episodes.head(10)

ICD10_CHAPTER
Digestive      33490
Circulatory    33285
Endocrine      33225
Name: count, dtype: int64

In [36]:
episodes_cost = med.groupby('ICD10_CHAPTER')['MEDICAL_PAID_AMOUNT'].sum().reset_index()
episodes_cost.sort_values(by='MEDICAL_PAID_AMOUNT', ascending=False)

,ICD10_CHAPTER,MEDICAL_PAID_AMOUNT
2,Endocrine,6703577.36
1,Digestive,6692078.46
0,Circulatory,6655438.42


In [37]:
episodes_to_keep = ['Diseases of the musculoskeletal system and connective tissue', 'Neoplasms', 'Diseases of the digestive system', 
                    'Diseases of the circulatory system', 'Diseases of the circulatory system', 'Diseases of the genitourinary system',
                   'Diseases of the nervous system', 'Mental, Behavioral and Neurodevelopmental disorders']
med1 = med
med1['ICD10_CHAPTER'] = med['ICD10_CHAPTER'].apply(lambda x: x if x in episodes_to_keep else 'OTHER')

In [38]:
med_meg = pd.pivot_table(
    med1,
    values='MEDICAL_PAID_AMOUNT',
    index='MEMBER_ID',
    columns='ICD10_CHAPTER',
    aggfunc='sum',
    fill_value=0
)
med_meg.reset_index()
med_meg.columns.name = None
med_meg.drop(columns='OTHER', inplace=True)
med_meg

""
MEMBER_ID
10000
10001
10002
10003
10004
...
19995
19996
19997


In [39]:
med_var = member1.merge(med_util, on='MEMBER_ID')
med_var = med_var.merge(med_meg, on='MEMBER_ID')
med_var.columns= med_var.columns.str.replace(' ', '_')
med_var

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
0,16252,OTHER,Subscriber,Male,56,Generation X,1194.55,309.93,913.87,377.89,...,538.23,-871.49,-61.82,-178.37,1,5,5,7,8,6
1,14684,NC,Domestic Partner,Male,73,Baby Boomers,665.93,803.29,1258.23,902.19,...,840.13,432.48,106.09,196.61,1,6,8,9,6,6
2,11731,OTHER,Spouse,Unknown,71,Baby Boomers,0.00,229.47,184.58,808.18,...,532.90,335.76,170.31,122.95,1,4,3,4,2,4
3,14742,OTHER,Subscriber,Female,55,Generation X,786.49,1075.99,442.21,943.05,...,315.10,-123.83,-28.21,10.03,1,7,7,6,6,7
4,14521,OTHER,Subscriber,Unknown,39,Millenials,321.49,92.41,461.79,121.04,...,410.82,-87.45,-17.55,-40.08,1,4,3,5,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15734,OTHER,Subscriber,Unknown,46,Generation X,719.24,117.49,14.26,339.34,...,245.43,-216.33,-46.85,-103.31,1,8,3,2,5,3
9996,15191,OTHER,Dependent,Male,15,Generation Z,211.22,557.73,505.10,0.00,...,804.78,-49.56,-5.80,23.60,1,6,4,6,4,7
9997,15390,OTHER,Subscriber,Unknown,43,Millenials,587.74,109.96,358.81,272.64,...,239.64,-200.97,-45.61,-46.85,1,5,2,4,2,2
9998,10860,OTHER,Dependent,Unknown,42,Millenials,27.67,538.65,176.45,249.54,...,257.72,166.52,182.58,23.32,1,4,4,3,5,4


In [45]:
mbr = members_20['MEMBER_ID'].unique()
members20_df = med_var[med_var['MEMBER_ID'].isin(mbr)]
members20_df

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
30,10321,OTHER,Spouse,Male,28,Generation Z,0.00,288.29,69.73,496.81,...,470.58,394.64,519.67,128.91,1,6,3,3,4,5
38,13973,TX,Dependent,Unknown,62,Baby Boomers,11.62,741.17,0.00,546.94,...,0.00,-93.49,-100.00,13.44,1,9,3,5,5,4
42,18362,PA,Subscriber,Female,21,Generation Z,1229.12,0.00,331.79,190.19,...,182.56,28.32,18.36,-36.47,1,2,2,4,3,2
128,19149,OTHER,Spouse,Male,39,Millenials,412.32,0.00,0.00,29.25,...,0.00,0.00,NaN,-58.21,1,1,1,2,1,3
138,11512,PA,Subscriber,Unknown,52,Generation X,349.40,361.24,789.11,639.69,...,738.84,738.84,0.00,203.58,1,5,6,5,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9855,12693,OTHER,Domestic Partner,Male,30,Millenials,1085.44,0.00,0.00,167.04,...,763.42,506.94,197.65,150.48,1,5,4,5,3,6
9915,12062,OTHER,Domestic Partner,Female,40,Millenials,530.42,369.09,0.00,216.13,...,485.19,457.60,1658.66,137.28,1,1,3,2,2,3
9932,19467,OTHER,Subscriber,Unknown,24,Generation Z,0.00,422.92,0.00,368.88,...,557.29,334.68,150.35,132.66,1,2,2,3,3,1
9946,11585,OTHER,Unknown,Female,16,Generation Z,228.02,697.20,67.05,0.00,...,403.01,321.16,392.38,120.73,1,3,3,3,3,3


In [46]:
med_var.columns

Index(['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope',
       'DX_IS_CHRONIC', 'ER_VISIT_FLAG', 'IS_PCP_VISIT',
       'IS_URGENT_CARE_VISIT', 'IS_PREVENTIVE_VISIT', 'IP_ADMIT_INC'],
      dtype='object')

In [47]:
members20_df.describe()

,MEMBER_ID,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,...,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope,DX_IS_CHRONIC,ER_VISIT_FLAG,IS_PCP_VISIT,IS_URGENT_CARE_VISIT,IS_PREVENTIVE_VISIT,IP_ADMIT_INC
count,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,...,344.00,344.00,335.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00
mean,14947.43,45.93,458.79,397.41,404.02,375.49,-58.12,150.52,-24.33,360.97,...,251.65,-109.32,198.57,-37.26,0.99,3.81,3.74,3.83,3.68,3.70
std,2774.14,19.76,462.80,393.76,435.73,379.63,505.67,843.52,184.13,383.61,...,252.23,462.31,1695.61,143.36,0.11,1.87,1.84,1.83,1.72,1.83
min,10053.00,13.00,0.00,0.00,0.00,0.00,-1993.62,-99.78,-767.83,0.00,...,0.00,-1864.71,-100.00,-573.28,0.00,0.00,0.00,0.00,0.00,0.00
25%,12631.25,28.00,99.99,75.00,56.14,73.48,-272.81,-46.40,-142.35,65.19,...,58.80,-286.62,-82.87,-109.93,1.00,3.00,2.00,3.00,3.00,2.00
50%,14897.00,45.00,333.25,329.11,263.55,277.16,0.00,0.00,-10.31,262.31,...,191.60,-35.86,-20.23,-21.81,1.00,4.00,4.00,4.00,4.00,4.00
75%,17233.00,62.00,702.76,579.76,669.24,548.23,198.57,83.13,103.06,523.27,...,368.20,156.62,33.05,41.96,1.00,5.00,5.00,5.00,5.00,5.00
max,19988.00,79.00,2751.90,1786.79,2262.92,2770.73,2184.95,11699.68,594.50,1864.71,...,1397.24,1161.58,28058.40,348.48,1.00,13.00,10.00,11.00,9.00,9.00


In [48]:
costs = members65_df[['MEMBER_ID', 'HOME_STATE', 'MEMBER_RELATIONSHIP', 'GENDER', 'Age',
       'GENERATIONS', 'Med_2021', 'Med_2022', 'Med_2023', 'Med_2024',
       'Med_Delta', 'Med_%_Change', 'Med_Slope', 'Rx_2021', 'Rx_2022',
       'Rx_2023', 'Rx_2024', 'Rx_Delta', 'Rx_%_Change', 'Rx_Slope']]
costs

,MEMBER_ID,HOME_STATE,MEMBER_RELATIONSHIP,GENDER,Age,GENERATIONS,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
30,10321,OTHER,Spouse,Male,28,Generation Z,0.00,288.29,69.73,496.81,0.00,0.00,127.19,75.94,0.00,105.23,470.58,394.64,519.67,128.91
38,13973,TX,Dependent,Unknown,62,Baby Boomers,11.62,741.17,0.00,546.94,535.32,4606.58,86.48,93.49,75.72,490.59,0.00,-93.49,-100.00,13.44
42,18362,PA,Subscriber,Female,21,Generation Z,1229.12,0.00,331.79,190.19,-1038.93,-84.53,-278.50,154.24,589.78,140.09,182.56,28.32,18.36,-36.47
128,19149,OTHER,Spouse,Male,39,Millenials,412.32,0.00,0.00,29.25,-383.07,-92.91,-114.92,0.00,693.35,111.24,0.00,0.00,NaN,-58.21
138,11512,PA,Subscriber,Unknown,52,Generation X,349.40,361.24,789.11,639.69,290.29,83.08,129.88,0.00,266.00,85.27,738.84,738.84,0.00,203.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9855,12693,OTHER,Domestic Partner,Male,30,Millenials,1085.44,0.00,0.00,167.04,-918.40,-84.61,-275.52,256.48,197.92,181.91,763.42,506.94,197.65,150.48
9915,12062,OTHER,Domestic Partner,Female,40,Millenials,530.42,369.09,0.00,216.13,-314.29,-59.25,-131.20,27.59,0.00,0.00,485.19,457.60,1658.66,137.28
9932,19467,OTHER,Subscriber,Unknown,24,Generation Z,0.00,422.92,0.00,368.88,0.00,0.00,68.37,222.61,0.00,322.58,557.29,334.68,150.35,132.66
9946,11585,OTHER,Unknown,Female,16,Generation Z,228.02,697.20,67.05,0.00,0.00,0.00,-131.42,81.85,0.00,243.83,403.01,321.16,392.38,120.73


In [49]:
costs.describe()

,MEMBER_ID,Age,Med_2021,Med_2022,Med_2023,Med_2024,Med_Delta,Med_%_Change,Med_Slope,Rx_2021,Rx_2022,Rx_2023,Rx_2024,Rx_Delta,Rx_%_Change,Rx_Slope
count,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,344.00,335.00,344.00
mean,14947.43,45.93,458.79,397.41,404.02,375.49,-58.12,150.52,-24.33,360.97,348.78,304.15,251.65,-109.32,198.57,-37.26
std,2774.14,19.76,462.80,393.76,435.73,379.63,505.67,843.52,184.13,383.61,356.61,320.83,252.23,462.31,1695.61,143.36
min,10053.00,13.00,0.00,0.00,0.00,0.00,-1993.62,-99.78,-767.83,0.00,0.00,0.00,0.00,-1864.71,-100.00,-573.28
25%,12631.25,28.00,99.99,75.00,56.14,73.48,-272.81,-46.40,-142.35,65.19,68.01,51.43,58.80,-286.62,-82.87,-109.93
50%,14897.00,45.00,333.25,329.11,263.55,277.16,0.00,0.00,-10.31,262.31,248.62,225.23,191.60,-35.86,-20.23,-21.81
75%,17233.00,62.00,702.76,579.76,669.24,548.23,198.57,83.13,103.06,523.27,521.42,452.36,368.20,156.62,33.05,41.96
max,19988.00,79.00,2751.90,1786.79,2262.92,2770.73,2184.95,11699.68,594.50,1864.71,2437.22,2101.36,1397.24,1161.58,28058.40,348.48


average med+rx net per member, normalized for 2024 = $11,834-1,554 = $10,280. Since 2021, employers have spent $10,280 average per member on those that have prescriptions for Wegovy, Saxenda or Zepbound for 3 years

In [50]:
members20_df.to_csv('glp1_members_pdc20.csv', index=False)

In [51]:
drugs = rx.groupby(['ART_DRUGGRPS_L1', 'ART_DRUGGRPS_L2', 'DRUG_NAME_PREFERRED'])['RX_PAID_AMOUNT'].sum().reset_index()
drugs.sort_values(by='RX_PAID_AMOUNT', ascending=False, inplace=True)
drugs

,ART_DRUGGRPS_L1,ART_DRUGGRPS_L2,DRUG_NAME_PREFERRED,RX_PAID_AMOUNT
28,Antidiabetics,GLP-1 Agonists,Wegovy,203655.05
89,Obesity Agents,ACE Inhibitors,Zepbound,202670.73
38,Antidiabetics,Inhalers,Wegovy,197866.23
149,Respiratory Agents,GLP-1 Agonists,Zepbound,197233.72
118,Obesity Agents,Inhalers,Wegovy,196570.97
...,...,...,...,...
57,Antihypertensives,Beta Blockers,Ventolin,46437.29
76,Antihypertensives,Inhalers,Synthroid,45791.58
44,Antihypertensives,ACE Inhibitors,Prinivil,45505.61
26,Antidiabetics,GLP-1 Agonists,Synthroid,43939.64


In [52]:
med.to_csv('GLP-1_Medical_PDC20.csv', index=False)